In [10]:
from scipy.io.wavfile import read as wavread
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, matthews_corrcoef
from sklearn.cross_validation import train_test_split, KFold, StratifiedKFold
import pandas as pd
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from scipy.signal import argrelextrema
import seaborn
import pickle
from copy import deepcopy
from sklearn.svm import LinearSVC
import tensorflow as tf
from keras.utils import np_utils

In [11]:
#DATA PATHS
sick_path = "./data/LungDiagnostics-sorted/_sick/"
nonsick_path = "./data/LungDiagnostics-sorted/_not-sick/"
val_path = "./data/LungDiagnostics-sorted/new_data/"

In [12]:
#PREPROCESS
def ffilter(X):
    new_X = []
    step = 8000
    for i in range(0, len(X), step):
        new_X = new_X + [np.mean(X[i:min(i+step, len(X))])]*step
    return np.array(new_X)

def get_data(path):
    d = wavread(path)
    rate = d[0]
    sound = d[1]
    return sound, rate

def smooth(sound_copy, quantile = 40):
    sound_copy[sound_copy < 0] = 0
    sound_copy_smoothed = ffilter(sound_copy)
    sound_copy_smoothed[sound_copy_smoothed < np.percentile(sound_copy_smoothed, quantile)] = 0
    return sound_copy_smoothed
    
def find_biggest_peaks(sound_copy_smoothed, count = 2):
    dots_begin = []
    dots_end = []
    for i in range(1, len(sound_copy_smoothed)-1):
        if (sound_copy_smoothed[i] == 0 and sound_copy_smoothed[i + 1] > 0):
            dots_begin.append(i)
        if (sound_copy_smoothed[i] > 0 and sound_copy_smoothed[i + 1] == 0):
            if (len(dots_begin) == len(dots_end)):
                dots_begin.append(0)
            dots_end.append(i)
    if (len(dots_begin) > len(dots_end)):
        dots_end.append(len(sound_copy_smoothed))
    dots_begin = np.array(dots_begin)
    dots_end = np.array(dots_end)
    peak_lens = dots_end - dots_begin
    biggest_peaks = np.argsort(peak_lens)[::-1][:2]
    if (len(biggest_peaks) < 2):
        return None
    else:
        result = []
        for i in biggest_peaks:
            result.append([dots_begin[i], dots_end[i]])
        return result
    
def extract_respiratory_cycle(path):
    data = []
    target = []
    rates = []
    errors = 0
    for f in os.listdir(path):
        if ('wav' not in f):
            continue
        if (('not' in f) or ('non' in f)):
            label = 0
        else:
            label = 1
        #Read sound data and rate
        sound, rate = get_data(path + f)
        sound_copy = deepcopy(sound)
        #Smooth sound
        sound_copy_smoothed = smooth(sound_copy, 40)
        #Find peaks
        dots = find_biggest_peaks(sound_copy_smoothed)
        if (dots == None):
            errors += 1
        else:
            sample = np.concatenate([sound[d[0]:d[1]] for d in dots])
            data.append(sample)
            rates.append(rate)
            target.append(label)
    print("Errors count {}".format(errors))
    return np.array(data), np.array(target), np.array(rates)

# data_val, target_val, rates_val = extract_respiratory_cycle(val_path)
# data_sick, target_sick, rates_sick = extract_respiratory_cycle(sick_path)
# data_nsick, target_nsick, rates_nsick = extract_respiratory_cycle(nonsick_path)

In [13]:
data_val, target_val, rates_val = pickle.load(open("./data/preprocessed/val_data.pkl", 'rb'))
data_sick, target_sick, rates_sick = pickle.load(open("./data/preprocessed/sick_data.pkl", 'rb'))
data_nsick, target_nsick, rates_nsick = pickle.load(open("./data/preprocessed/nsick_data.pkl", 'rb'))

data_train = np.concatenate([data_sick, data_nsick])
target_train = np.concatenate([target_sick, target_nsick])
rates_train = np.concatenate([rates_sick, rates_nsick])

In [6]:
#FEATURE EXTRACTION
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
from PIL import Image
import PIL

mfcc_train = []
mfcc_val = []
numcep = 13
height = 350
for d,r in zip(data_train, rates_train):
    (rate,sig) = r, d
    mfcc_feat = mfcc(sig,rate, numcep=numcep)
    d_mfcc_feat = delta(mfcc_feat, 2)
    d_mfcc_zip = np.array(Image.fromarray(d_mfcc_feat).resize((numcep, height), PIL.Image.NEAREST))
    mfcc_train.append(d_mfcc_zip)
mfcc_train = np.array(mfcc_train)

for d,r in zip(data_val, rates_val):
    (rate,sig) = r, d
    mfcc_feat = mfcc(sig,rate, numcep=numcep)
    d_mfcc_feat = delta(mfcc_feat, 2)
    d_mfcc_zip = np.array(Image.fromarray(d_mfcc_feat).resize((numcep, height), PIL.Image.NEAREST))
    mfcc_val.append(d_mfcc_zip)
mfcc_val = np.array(mfcc_val)

In [7]:
mfcc_train_flatten = np.array([x.flatten() for x in mfcc_train])
mfcc_val_flatten = np.array([x.flatten() for x in mfcc_val])

In [14]:
#MODEL EVALUATION
def get_model_by_name(model_name):
    models = {"LogisticRegression" : LogisticRegression(), 'RandomForest' : RandomForestClassifier(n_jobs=4, n_estimators=100), 
           'GBM' : GradientBoostingClassifier(), 'SVM': LinearSVC()}
    if (model_name in models):
        return models[model_name]
    else:
        print("Illegal model name")
        
def calc_metrics(true, pred, pref):
    acc = accuracy_score(true, pred.round())
    print("{} Accuracy: {}".format(pref, np.round(acc,3)))
    f1 = f1_score(true, pred.round())
    print("{} F1: {}".format(pref, np.round(f1,3)))
    mttc = matthews_corrcoef(true, pred.round())
    print("{} MTTC: {}".format(pref, np.round(mttc,3)))
    cm = confusion_matrix(true, pred.round())
    print("{} CM: {}".format(pref, cm))

def train_model(X_train, y_train, X_test, y_test, model_name):
    model = get_model_by_name(model_name)
    model.fit(X_train, y_train)
    test_predicts = model.predict(X_test)
    calc_metrics(y_test, test_predicts, "Test")
    return model

X_train, X_test, y_train, y_test = train_test_split(mfcc_train_flatten, target_train, test_size=0.25, random_state=23)

for model_name in ['LogisticRegression', 'RandomForest', 'GBM', 'SVM']:
    print(model_name)
    model = train_model(X_train, y_train, X_test, y_test, model_name)
    val_predicts = model.predict(mfcc_val_flatten)
    calc_metrics(target_val, val_predicts, "Validation")
    print("-"*15)

LogisticRegression
Test Accuracy: 0.817
Test F1: 0.884
Test MTTC: 0.482
Test CM: [[14  5]
 [16 80]]
Validation Accuracy: 0.714
Validation F1: 0.8
Validation MTTC: 0.471
Validation CM: [[ 3  6]
 [ 0 12]]
---------------
RandomForest
Test Accuracy: 0.957
Test F1: 0.975
Test MTTC: 0.837
Test CM: [[14  5]
 [ 0 96]]
Validation Accuracy: 0.571
Validation F1: 0.727
Validation MTTC: 0.0
Validation CM: [[ 0  9]
 [ 0 12]]
---------------
GBM
Test Accuracy: 0.957
Test F1: 0.975
Test MTTC: 0.837
Test CM: [[14  5]
 [ 0 96]]
Validation Accuracy: 0.571
Validation F1: 0.727
Validation MTTC: 0.0
Validation CM: [[ 0  9]
 [ 0 12]]
---------------
SVM
Test Accuracy: 0.826
Test F1: 0.89
Test MTTC: 0.497
Test CM: [[14  5]
 [15 81]]
Validation Accuracy: 0.714
Validation F1: 0.8
Validation MTTC: 0.471
Validation CM: [[ 3  6]
 [ 0 12]]
---------------


In [ ]:
y_train = np.reshape(y_train, (y_train.shape[0], 1))
y_test = np.reshape(y_test, (y_test.shape[0], 1))

In [130]:
nn = NN(X_train.shape[1])
nn.fit(X_train, y_train, X_test, y_test, training_epochs=50)
test_predicts = nn.predict(X_test)
calc_metrics(y_test, test_predicts, "Test")
val_predicts = nn.predict(mfcc_val_flatten)
calc_metrics(target_val, val_predicts, "Validation")

Test Accuracy: 0.548
Test F1: 0.662
Test MTTC: 0.121
Test CM: [[12  7]
 [45 51]]
Validation Accuracy: 0.667
Validation F1: 0.774
Validation MTTC: 0.375
Validation CM: [[ 2  7]
 [ 0 12]]


In [131]:
class NN(object):       
    def __init__(self, n_input, n_hidden_1 = 500, n_hidden_2 = 500, n_classes = 1):
        self.display_step = 1
        self.n_input = n_input
        self.n_hidden_1 = n_hidden_1
        self.n_hidden_2 = n_hidden_2
        self.n_classes = n_classes
        # Store layers weight & bias
        weights = {
            'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
            'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
            'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
        }
        biases = {
            'b1': tf.Variable(tf.random_normal([n_hidden_1])),
            'b2': tf.Variable(tf.random_normal([n_hidden_2])),
            'out': tf.Variable(tf.random_normal([n_classes]))
        }
        # tf Graph input
        self.x = tf.placeholder("float", [None, self.n_input])
        self.y = tf.placeholder("float", [None, self.n_classes])
        # Construct model
        self.pred = self.multilayer_perceptron(self.x, weights, biases)
     
    def multilayer_perceptron(self, x, weights, biases):
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        # Hidden layer with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        # Output layer with linear activation
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
        out_layer = tf.nn.sigmoid(out_layer)
        return out_layer

    def fit(self, X_train = None, y_train = None, X_test = None, y_test = None, training_epochs = 10, batch_size = 16, display_step = 1,learning_rate = 0.01):
        # Define loss and optimizer
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.pred, labels=self.y))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
        saver = tf.train.Saver()
        # Initializing the variables
        self.init = tf.global_variables_initializer()
        # Launch the graph
        with tf.Session() as sess:
            sess.run(self.init)
            # Training cycle
            for epoch in range(training_epochs):
                total_batch = int(len(X_train)/batch_size)
                # Loop over all batches
                for i in range(total_batch):
                    batch_x = X_train[i*batch_size:(i+1)*batch_size]
                    batch_y = y_train[i*batch_size:(i+1)*batch_size]
                    sess.run([optimizer, cost], feed_dict={self.x: batch_x,
                                                                  self.y: batch_y})
            
    def predict(self, X_test=None):
        with tf.Session() as sess:
            sess.run(self.init)
            pr = sess.run(self.pred, feed_dict={self.x: X_test})
        return pr